# Loading the Data

Loading the data with help of the `ExperimentDataLoaderWidget`. 

You can select which experiments to load, define a different name (the 'name' field, not the 'short name' field) and resort the data. The name and the order of the experiments influence the legends of the plots.

Here we also compute an extra statistic, the total reward of each experiment / repetition, using the custom `cal_total_reward` function.

You have to press the 'Load Data' button to load the data.

If you don't see a table with the experiments, then you might not have installed the required qgrid widget. Please see the installation instructions on the main page of the exputils. 

In [1]:
# load data

# import the experiment utilities package
import exputils as eu
import numpy as np

def cal_total_reward(event_descr):
    
    experiment_data = event_descr['owner'].experiment_data
    
    for single_exp_data in experiment_data.values():
        for single_rep_data in single_exp_data.repetition_data.values():
            
            single_rep_data.total_reward = np.nansum(single_rep_data.reward_per_episode)     



# create an experiment data loader, by default it will load data from '../experiments'
experiment_data_loader = eu.gui.jupyter.ExperimentDataLoaderWidget()

# add calculatation of extra statistics 
#experiment_data_loader.on_experiment_data_loaded(cal_total_reward)

display(experiment_data_loader)
experiment_data_loader.load_data()

ExperimentDataLoaderWidget(children=(Box(children=(Button(description='Update Descriptions', layout=Layout(hei…

Output()

We can access the data via the `experiment_data` property of the `ExperimentDataLoaderWidget`.

The data is composed of dictionaries that hold the individual data for each repetition.

# Plotting the Data

The exputils have several builtin plotting functions. These can be accessed via the `ExperimentDataPlotSelectionWidget`.

Write in the 'Data Sources' field which data you would like to plot. Then you can select a plotting function and change its configuration.

Examples: 
 * plotting the 'reward_per_episode' with the 'plotly_meanstd_scatter' plotter shows the performance of each experiment in a line graph
 * plotting the 'total_reward' with the 'plotly_box' or 'plotly_meanstd_bar' compares the total reward that was collected during each reward
 
If you want to 'save' a plot, to use it later again, then use the code production to generate a cell below the `ExperimentDataPlotSelectionWidget` that has the code to plot the figure again. The cells in the subchapters 'Total Reward' and 'Reward per Episode' at the end of this notebook were generated by this method. After saving the jupyter notebook all your configurations will be saved.

In [2]:
# experiment data selection plotter that takes as input the data loader to plot its loaded data
experiment_data_plotter = eu.gui.jupyter.ExperimentDataPlotSelectionWidget(experiment_data_loader)
display(experiment_data_plotter)

/opt/homebrew/lib/python3.11/site-packages/exputils/gui/jupyter/experiment_ids_selection_widget.py:108: UserWarning: Experiment id '000011' does not exists in list of experiment ID's. Setting will be ignored.
  warnings.warn('Experiment id {!r} does not exists in list of experiment ID\'s. Setting will be ignored.'.format(exp_id))
/opt/homebrew/lib/python3.11/site-packages/exputils/gui/jupyter/experiment_ids_selection_widget.py:108: UserWarning: Experiment id '000012' does not exists in list of experiment ID's. Setting will be ignored.
  warnings.warn('Experiment id {!r} does not exists in list of experiment ID\'s. Setting will be ignored.'.format(exp_id))


ExperimentDataPlotSelectionWidget(children=(HBox(children=(Label(value='Data Sources:', layout=Layout(min_widt…

Output()

### Observation: 

We introduced a worse initial state... Which it takes many epochs to recover from....
URBF makes learning slower but more stable...


## Mean value

In [30]:
import numpy as np
import plotly.graph_objects as go

data = experiment_data_loader.experiment_data['000011']['repetition_data'][0]


means = []
for mean_idx in range(16):
    means.append(data[f"mean{mean_idx}"])

    
data = np.stack(means, axis=1)
print(data.shape)
#data = np.expand_dims(data, axis=1)

# Preparing data for plotting
epochs = np.arange(1, data.shape[0])
traces = []
for i in range(data.shape[1]):
    traces.append(go.Scatter(x=epochs, y=data[:, i], mode='lines', name=f'Value {i+1}'))

# Plotting with Plotly
layout = go.Layout(title='Change of 16 Values over 500 Steps',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict(title='Value'),
                   template='plotly_dark')

fig = go.Figure(data=traces, layout=layout)
fig.show()


(500, 16)


1. Base config: Means are barely moving as they should... 
    - The concentration should be in (-5,5) but this does not seem to work out... 
    - There the function is mostly zero which can be produced by the following layers no need to adjust there... 
    - We need a stronger signal for mean movement...
    - The strongest change in mean AND var is at 4!
    - Backprop does not send the right signal to improve
2. Lr Test: Introduction of different Learning rates...
    - 

## Vars


In [14]:
import numpy as np
import plotly.graph_objects as go


data = experiment_data_loader.experiment_data['000001']['repetition_data'][1]


means = []
for mean_idx in range(16):
    means.append(data[f"var{mean_idx}"])

    
data = np.stack(means, axis=1)

# Preparing data for plotting
epochs = np.arange(1, data.shape[0])
traces = []
for i in range(data.shape[1]):
    traces.append(go.Scatter(x=epochs, y=data[:, i], mode='lines', name=f'Value {i+1}'))

# Plotting with Plotly
layout = go.Layout(title='Change of 16 Values over 500 Steps',
                   xaxis=dict(title='Epoch'),
                   yaxis=dict(title='Value'),
                   template='plotly_dark')

fig = go.Figure(data=traces, layout=layout)
fig.show()